In [1]:
import pandas as pd
import numpy as np
import gensim
from nltk.stem.porter import *
np.random.seed(2020)
from gensim import corpora, models
import collections
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [12]:
top_20 = pd.read_csv('Top 20.csv')
top_20.head()

,Id,Name,Screen Name,Location,Followers Count,Friends Count,Listed Count,Created At,Favourites Count,Verified,Tweets Count,Tweet Count,Average Likes,Average comments,Average retweets,Influence Matrix
0,9.640000e+17,Mrs. Krassenstein,HKrassenstein,"Fort Myers, FL",193660,5333,696,2/14/2018 15:03,8193,False,12724,567,15130.511460,673.735450,1485.613757,0.193184
1,8.220000e+17,The White House,WhiteHouse,"Washington, D.C.",25864844,14,18148,1/19/2017 22:54,20,True,23641,478,12244.142260,1847.035565,3425.449791,0.266669
2,1.113425e+07,GOP,GOP,NaN,2706710,1650,11202,12/13/2007 17:15,264,True,44452,462,4947.019481,1140.647186,1453.670996,0.258233
3,2.353606e+09,Ronna McDaniel,GOPChairwoman,NaN,1018735,2580,3617,2/20/2014 18:05,8540,True,16815,322,9220.341615,1270.540373,2974.801242,0.212354
4,2.105926e+07,Ted Lieu,tedlieu,California,1501775,10224,7460,2/17/2009 3:12,76096,True,27541,242,13269.834710,586.619835,4103.318182,0.184236


In [13]:
top_20 = top_20[:20]
top_20

,Id,Name,Screen Name,Location,Followers Count,Friends Count,Listed Count,Created At,Favourites Count,Verified,Tweets Count,Tweet Count,Average Likes,Average comments,Average retweets,Influence Matrix
0,9.640000e+17,Mrs. Krassenstein,HKrassenstein,"Fort Myers, FL",193660,5333,696,2/14/2018 15:03,8193,False,12724,567,15130.511460,673.735450,1485.613757,0.193184
1,8.220000e+17,The White House,WhiteHouse,"Washington, D.C.",25864844,14,18148,1/19/2017 22:54,20,True,23641,478,12244.142260,1847.035565,3425.449791,0.266669
2,1.113425e+07,GOP,GOP,NaN,2706710,1650,11202,12/13/2007 17:15,264,True,44452,462,4947.019481,1140.647186,1453.670996,0.258233
3,2.353606e+09,Ronna McDaniel,GOPChairwoman,NaN,1018735,2580,3617,2/20/2014 18:05,8540,True,16815,322,9220.341615,1270.540373,2974.801242,0.212354
4,2.105926e+07,Ted Lieu,tedlieu,California,1501775,10224,7460,2/17/2009 3:12,76096,True,27541,242,13269.834710,586.619835,4103.318182,0.184236
5,2.558126e+08,Don Winslow,donwinslow,Repped by The Story Factory,540001,18502,2579,2/22/2011 2:58,168471,True,59151,235,17755.455320,690.144681,5531.238298,0.182893
6,7.300000e+17,Team Trump,TeamTrump,USA,2419868,132,4972,5/9/2016 14:15,3524,True,30313,200,19858.160000,1764.505000,5825.825000,0.174415
7,9.390910e+05,Joe Biden,JoeBiden,"Wilmington, DE",20184162,30,29631,3/11/2007 17:51,20,True,6869,198,109180.298000,7224.146465,21769.181820,0.200926
8,2.220376e+07,Mike Pence,Mike_Pence,NaN,5930027,46,11749,2/27/2009 23:04,742,True,12837,182,19825.269230,1843.489011,3970.203297,0.168253
9,9.312863e+08,#ThePersistence,ScottPresler,Make Kindness Cool Again,902658,39818,2484,11/7/2012 5:23,333116,True,82193,165,14538.181820,687.921212,3428.727273,0.158698


In [14]:
data = pd.read_csv("Combined Information.csv")
data = data[data['Screen Name'].isin(list(top_20['Screen Name']))]
print(data.shape)
data.head()

(3604, 19)


,index,Screen Name,Date,Tweet,Likes,Comments,Retweets,Mentions,Clean Text,Id,Name,Location,Followers Count,Friends Count,Listed Count,Created At,Favourites Count,Verified,Tweets Count
0,0,donwinslow,27-Nov,Dear @realDonaldTrump President-elect Biden's...,15800,173,2300,@realDonaldTrump,dear president elect national popular vote lea...,255812611.0,Don Winslow,Repped by The Story Factory,540001,18502,2579,2/22/2011 2:58,168471,True,59151
1,81,donwinslow,27-Nov,Dear @realDonaldTrump This is incredibly s...,16300,325,1900,@realDonaldTrump,dear incredibly simple lost,255812611.0,Don Winslow,Repped by The Story Factory,540001,18502,2579,2/22/2011 2:58,168471,True,59151
2,94,donwinslow,26-Nov,Dear @realDonaldTrump It's going to be a ve...,9600,172,935,@realDonaldTrump,dear going different,255812611.0,Don Winslow,Repped by The Story Factory,540001,18502,2579,2/22/2011 2:58,168471,True,59151
3,151,donwinslow,26-Nov,I'm thankful to all the people who didn't trav...,62100,813,4300,NaN,thankful people travel thanksgiving thankful t...,255812611.0,Don Winslow,Repped by The Story Factory,540001,18502,2579,2/22/2011 2:58,168471,True,59151
4,234,donwinslow,25-Nov,What James Comey said about Michael Flynn was ...,9500,213,1500,NaN,said true told trump true broke law trump brok...,255812611.0,Don Winslow,Repped by The Story Factory,540001,18502,2579,2/22/2011 2:58,168471,True,59151


In [15]:
data.dropna(subset=['Clean Text'],inplace=True)
data['Clean Text'].isna().sum()

0

In [16]:
from nltk.corpus import words

text = words.words()
wordlist = [w for w in text if len(w) >1]
i=0
for doc in data['Clean Text']:
    data['Clean Text'][i] = " ".join(word for word in doc.split() if word in wordlist)
    i = i+1

C:\Users\saksh\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [17]:
def topic_def(df):
    doc_clean = [str(doc).split() for doc in df['Clean Text']] 
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    doc_tfidf = models.TfidfModel(doc_term_matrix)

    doc_tfidf_matrix = []
    for i in range(len(df['Clean Text'])):
        doc_tfidf_matrix.append(doc_tfidf[doc_term_matrix[i]])

    topic = 5
    words=10
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(corpus=doc_tfidf_matrix, 
                   id2word=dictionary,
                    num_topics=topic,  
                    iterations=100,
                   random_state=100,
                   update_every=1,
                   chunksize=100,
                   passes=10,
                   alpha='auto',
                    per_word_topics=True)
    topics = ldamodel.print_topics(num_topics=topic, num_words=words)
    
    return topics,topic

In [18]:
def output_df(topics,author,topic):
    topic_list =[]
    topic_weight =[]
    topics_weight =[]

    for row in topics:
        topic_list.append(str(row).split('+'))
    for row in topic_list:
        topic_weight.append(re.sub('[^A-Za-z0-9.,''* ]', ' ', str(row)))
    for row in topic_weight:
        topics_weight.append(str(row).split(','))

    dicts = pd.DataFrame(topics_weight)
    
    dicts = dicts.rename(columns={0: "index"})
    dicts = dicts.drop(['index'], axis=1)

    i = 1
    for col_name in dicts.columns: 
        col_name = 'word'+'_{}'.format(i)
        dicts = dicts.rename(columns={i: col_name})
        i = i + 1

    j=1
    k=1
    for col_name in dicts.columns:
        temp=[]
        for i in range(0,topic):
            temp.append(str(dicts[col_name][i]).split("* ")[0])
            dicts[col_name][i] = str(dicts[col_name][i]).split("* ")[1]
        name = 'score'+'_{}'.format(j)
        j=j+1
        dicts.insert(k,name,temp)
        k = k+2
    dicts['Author'] = author 
    return dicts

In [19]:
def topic_map(df1,top_20):
    dictionary = pd.DataFrame(columns=['word_1', 'score_1', 'word_2', 'score_2', 'word_3', 'score_3', 'word_4',
       'score_4', 'word_5', 'score_5', 'word_6', 'score_6', 'word_7',
       'score_7', 'word_8', 'score_8', 'word_9', 'score_9', 'word_10',
       'score_10', 'Author'])
    for auth in list(top_20['Screen Name']):
        df = df1[df1['Screen Name'] == auth]
        topics,topic = topic_def(df)
        output = output_df(topics,auth,topic)
        
        dictionary = dictionary.append(output)
        
    return dictionary
    

dictionary = topic_map(data,top_20)        

In [20]:
dictionary.shape

(100, 21)

In [21]:
dictionary.to_csv("Topics.csv",index=False)

In [22]:
dictionary


,word_1,score_1,word_2,score_2,word_3,score_3,word_4,score_4,word_5,score_5,...,score_6,word_7,score_7,word_8,score_8,word_9,score_9,word_10,score_10,Author
0,trump,0.019,president,0.015,people,0.009,vote,0.007,get,0.007,...,0.007,would,0.007,government,0.006,white,0.006,state,0.006,HKrassenstein
1,like,0.013,joe,0.009,time,0.008,link,0.008,united,0.008,...,0.007,day,0.007,come,0.007,republican,0.007,real,0.006,HKrassenstein
2,country,0.009,law,0.007,campaign,0.007,breaking,0.006,order,0.006,...,0.006,great,0.006,today,0.005,also,0.005,soldier,0.005,HKrassenstein
3,tweet,0.010,going,0.008,love,0.008,ballot,0.007,never,0.006,...,0.006,quote,0.006,racist,0.006,spoken,0.005,test,0.005,HKrassenstein
4,testing,0.008,democrat,0.008,view,0.007,fact,0.007,new,0.006,...,0.006,let,0.006,back,0.006,radical,0.006,best,0.006,HKrassenstein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,want,0.067,uninformed,0.067,cheerleader,0.067,sound,0.067,woodward,0.067,...,0.067,said,0.067,open,0.067,trump,0.067,president,0.067,JimGaffigan
1,sound,0.067,uninformed,0.067,want,0.067,woodward,0.067,cheerleader,0.067,...,0.067,trump,0.067,said,0.067,fancy,0.067,need,0.067,JimGaffigan
2,sound,0.156,uninformed,0.156,want,0.156,trump,0.082,said,0.041,...,0.041,country,0.041,president,0.041,cheerleader,0.041,fancy,0.041,JimGaffigan
3,sound,0.067,want,0.067,uninformed,0.067,cheerleader,0.067,fancy,0.067,...,0.067,said,0.067,trump,0.067,open,0.067,woodward,0.067,JimGaffigan


In [23]:
topics

NameError: name 'topics' is not defined